# Analysis

In this analysis, unit of time used in simulation is minutes.

Credit:

* Analysis of the warm-up period, number of replications and spread of results across replications is adapted from Tom Monks (2024) HPDM097 - Making a difference with health data (https://github.com/health-data-science-OR/stochastic_systems) (MIT License). 

License:

* This project is licensed under the MIT License. See the LICENSE file for more details.

TODO: Create Latex-formatted table.

TODO: Use consistent base parameters (choose some and stick with them).

TODO: Mention alternative methods for choosing warm-up period length (or keep it simple?)

TODO: Mention alternative methods for choosing number of replications (or keep it simple?)

TODO: Consider whether to add time unit to model docstrings etc.

In [1]:
# Model code
from model import Defaults, Trial

# Other dependencies
import numpy as np
import time
import itertools
from IPython.display import display
import pandas as pd
import plotly.express as px
import scipy.stats as st

## Choosing length of warm-up period

A suitable length for the warm-up period can be determined using the **time series inspection approach**. This involves looking at performance measures over time to identify when the system is exhibiting **steady state behaviour** (even though the system will never truly reach a "steady state").

If we simply plot the mean result at regular intervals, this would vary too much. Therefore, we plot the **cumulative mean** of the performance measure, and look for the point at which this **smoothes out and stabilises**. This indicates the point for the warm-up period to end.

This should be assessed when running the model using the base case parameters. If these change, you should reassess the appropriate warm-up period.

In [2]:
def time_series_inspection(data_collection_period, warm_up=None):
    """
    Time series inspection method for determining length of warm-up.

    Arguments:
        data_collection_period (float):
            Length of time to run the simulation for.
        warm_up (float, optional):
            Location on X axis to plot vertical red line indicating the chosen
            warm-up period. Defaults to None, which will not plot a line.
    """
    # Set model parameters
    param = Defaults()
    param.patient_inter = 4
    param.mean_n_consult_time = 10
    param.number_of_nurses = 5
    param.warm_up_period = 0
    param.data_collection_period = data_collection_period
    param.number_of_runs = 10
    param.audit_interval = 120
    # display(param.__dict__)

    # Run model
    choose_warmup = Trial(param)
    choose_warmup.run_trial()

    # Filter to nurse results
    nurse = choose_warmup.interval_audit_df[
        choose_warmup.interval_audit_df['resource_name'] == 'nurse']

    # Reformat so index is simulation time and columns are each replication/run
    reformat = (
        nurse[['simulation_time', 'perc_utilisation', 'run']]
        .pivot(index='simulation_time',
               columns='run',
               values='perc_utilisation'))
    # display(reformat.head())

    # Find mean at each time interval, and then calculate cumulative mean
    cumulative = reformat.mean(axis=1).expanding().mean()

    # Create plot
    fig = px.line(cumulative)
    fig.update_layout(
        xaxis_title = 'Run time (minutes)',
        yaxis_title = 'Cumulative mean nurse utilisation',
        showlegend=False
    )

    # If specified, add vertical line to indicate suggested warm-up length
    if warm_up is not None:
        fig.add_vline(x=warm_up, line_color='red', line_dash='dash')

    fig.show()

# TODO: Apply for utilisation and wait time

Having run the model for three days, it appears to reach a steady state at around 2500 minutes.

In [3]:
time_series_inspection(data_collection_period=1440*3, warm_up=2520)

However, it is important to look far ahead - so we run it for 20 days, and actually find it reaches a steady state at around 10,000 minutes.

In [4]:
time_series_inspection(data_collection_period=1440*20, warm_up=10080)

## Choosing the number of replications

The **confidence interval method** can be used to select the number of replications to run. The more replications you run, the narrower your confidence interval becomes, leading to a more precise estimate of the model's mean performance.

First, you select a desired confidence interval - for example, 95%. Then, run the model with an increasing number of replications, and identify the number required to achieve that precision in the estimate of a given metric - and also, to maintain that precision (as the intervals may converge or expand again later on).

This method is less useful for values very close to zero - so, for example, when using utilisation (which ranges from 0 to 1) it is recommended to multiple values by 100.

In [5]:
# TODO

## Spread of replication results

In [6]:
# Set model parameters
param = Defaults()
param.number_of_runs = 50
display(param.__dict__)

# Run trial
trial = Trial(param)
trial.run_trial()

{'_initialising': False,
 'patient_inter': 5,
 'mean_n_consult_time': 35,
 'number_of_nurses': 9,
 'warm_up_period': 0,
 'data_collection_period': 600,
 'number_of_runs': 50,
 'audit_interval': 5,
 'scenario_name': 0}

In [7]:
def plot_results_spread(column, x_label, y_label='Frequency'):
    """
    Plot spread of results from across replications, for chosen column.

    Arguments:
        column (str):
            Name of column to plot.
        x_label (str):
            X axis label.
        y_label (str):
            Y axis label
    """
    fig = px.histogram(trial.trial_results_df[column])
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label
    )
    fig.show()


plot_results_spread('arrivals', 'Arrivals')
plot_results_spread('mean_q_time_nurse', 'Mean wait time for nurse')
plot_results_spread('mean_time_with_nurse', 'Mean length of nurse consultation')
plot_results_spread('average_nurse_utilisation', 'Mean nurse utilisation')

## Other stuff

In [8]:
single_trial = Trial(param=Defaults())
single_trial.run_trial()
single_trial.trial_results_df

uncertainty_metrics = {}
for col in ['mean_q_time_nurse', 'average_nurse_utilisation']:
    # Calculate mean, standard deviation and count
    data = single_trial.trial_results_df[col]
    mean = data.mean()
    std_dev = data.std()
    n = len(data)

    # Calculate 95% confidence interval
    ci_lower, ci_upper = st.t.interval(
        confidence=0.95, df=len(data)-1, loc=mean, scale=st.sem(data))

    # Save as dictionary
    uncertainty_metrics[col] = {
        'mean': mean,
        'std_dev': std_dev,
        'lower_95_ci': ci_lower,
        'upper_95_ci': ci_upper
    }
# Convert to dataframe
pd.DataFrame(uncertainty_metrics)

,mean_q_time_nurse,average_nurse_utilisation
mean,6.628344,0.739750
std_dev,7.687874,0.070505
lower_95_ci,-2.917412,0.652207
upper_95_ci,16.174100,0.827293


In [9]:
# Run a single trial
single_trial = Trial(param=Defaults())
single_trial.run_trial()

# Preview results
display(single_trial.patient_results_df)
display(single_trial.trial_results_df)
display(single_trial.interval_audit_df)

# Plot interval audit utilisation
fig = px.line(single_trial.interval_audit_df,
              x='simulation_time', y='perc_utilisation', color='run')
fig.show()

# Calculate and plot median utilisation
interval_audits_median = (single_trial
                          .interval_audit_df
                          .drop('resource_name', axis=1)
                          .groupby('simulation_time')
                          .median()
                          .reset_index())
fig = px.line(interval_audits_median,
              x='simulation_time',
              y='perc_utilisation')
fig.show()


,patient_id,arrival_time,q_time_nurse,time_with_nurse,run
0,1,0.000000,0.0,28.108654,0
1,2,16.467639,0.0,13.370060,0
2,3,20.283293,0.0,12.746160,0
3,4,26.545128,0.0,18.531603,0
4,5,27.674876,0.0,97.593265,0
...,...,...,...,...,...
593,120,574.675966,NaN,NaN,4
594,121,578.946350,NaN,NaN,4
595,122,589.400743,NaN,NaN,4
596,123,591.362454,NaN,NaN,4


,run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,average_nurse_utilisation
0,0,0,115,0.260844,34.423436,0.715650
1,1,0,118,4.667636,35.376097,0.681010
2,2,0,115,5.412314,37.156265,0.724797
3,3,0,126,2.885792,33.429567,0.715003
4,4,0,124,19.915133,41.311555,0.862291


,resource_name,simulation_time,number_utilised,number_available,queue_length,run,perc_utilisation
0,nurse,0,0,9,0,0,0.000000
1,nurse,5,1,9,0,0,0.111111
2,nurse,10,1,9,0,0,0.111111
3,nurse,15,1,9,0,0,0.111111
4,nurse,20,2,9,0,0,0.222222
...,...,...,...,...,...,...,...
595,nurse,575,9,9,7,4,1.000000
596,nurse,580,9,9,6,4,1.000000
597,nurse,585,9,9,5,4,1.000000
598,nurse,590,9,9,4,4,1.000000


In [10]:
# Run with 1 to 14 cores
speed = []
param=Defaults()
param.number_of_runs = 100
for i in range(1, 15, 1):
    start_time = time.time()
    my_trial = Trial(param)
    my_trial.run_trial(cores=i)
    run_time = round((time.time() - start_time), 3)
    speed.append({'Cores': i, 'Run Time (seconds)': run_time})

# Display and plot time by number of cores
timing_results = pd.DataFrame(speed)
print(timing_results)
fig = px.line(timing_results, x='Cores', y='Run Time (seconds)')
fig.show()

    Cores  Run Time (seconds)
0       1               0.193
1       2               0.427
2       3               0.425
3       4               0.424
4       5               0.450
5       6               0.460
6       7               0.465
7       8               0.077
8       9               0.177
9      10               0.163
10     11               0.558
11     12               0.077
12     13               0.065
13     14               0.166


In [11]:
# TODO: Alter how this runs so fresh Defaults() each time

# Define a set of scenarios
param = Defaults()
param.number_of_runs = 5
scenarios = {
    'patient_inter': [5, 10, 15],
    'mean_n_consult_time': [15, 20, 35],
    'number_of_nurses': [3, 6, 9]
}

# Find every possible permutation of the scenarios
all_scenarios_tuples = list(itertools.product(*scenarios.values()))
# Convert back into dictionaries
all_scenarios_dicts = [
    dict(zip(scenarios.keys(), p)) for p in all_scenarios_tuples]
# Preview some of the scenarios
print(f'There are {len(all_scenarios_dicts)} scenarios. For example:')
display(all_scenarios_dicts[0:6])

# Run the scenarios...
results = []
for index, scenario_to_run in enumerate(all_scenarios_dicts):
    # Overwrite defaults from the passed dictionary
    param.scenario_name = index
    for key in scenario_to_run:
        setattr(param, key, scenario_to_run[key])
    # Run trial and keep trial-level results
    my_trial = Trial(param)
    my_trial.run_trial()
    results.append(my_trial.trial_results_df)
# View mean results by scenario
display(pd.concat(results)
        .drop('run_number', axis=1)
        .groupby('scenario')
        .mean()
        .head(20))

# TODO: Issue: warm-up patients use resources but their activity is excluded
# from metrics. Post-warm-up patients queue behind these, making it look
# like resources are under-utilised during the measurement period if there are
# long queues (e.g. due to really short inter-arrival times)

There are 27 scenarios. For example:


[{'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 15, 'number_of_nurses': 9},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 3},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 6},
 {'patient_inter': 5, 'mean_n_consult_time': 20, 'number_of_nurses': 9}]

,arrivals,mean_q_time_nurse,mean_time_with_nurse,average_nurse_utilisation
scenario,,,,
0,119.6,28.752656,15.572971,0.895605
1,119.6,0.650960,15.625492,0.506919
2,119.6,0.004197,15.576195,0.338975
3,119.6,76.038860,21.463783,0.974331
4,119.6,2.880780,20.790698,0.662758
5,119.6,0.105159,20.768260,0.446922
6,119.6,172.464534,38.317702,0.982788
7,119.6,42.185221,36.822320,0.941436
8,119.6,6.628344,36.339384,0.739750
